以下の順に実装する。
1. extract bbox area from single tiff
2. create bbox from aoi polygon
3. iterate over multiple tiff
4. without open archives

In [1]:
import rasterio
import rasterio.mask
from rasterio.plot import show
import os
import re
import pandas as pd
import geopandas as gpd
import gzip
import datetime

data_dir = '../../data/'

tiffs = [os.path.join(data_dir, 'ls', file) for file in ["LC08_L1TP_223067_20190829_20190903_01_T1_sr_band2.tif",
         "LC08_L1TP_223067_20190829_20190903_01_T1_sr_band3.tif", 
         "LC08_L1TP_223067_20190829_20190903_01_T1_sr_band4.tif"]]
aoi = os.path.join(data_dir, 'aoi', 'aoi.shp')


In [27]:
# Landsat/
meta_file = os.path.join(data_dir, 'ls', 'LANDSAT_8_C1.csv.gz')
archives = ["LC082230672019081301T1-SC20200511102436.tar.gz", 
           "LC082230672019082901T1-SC20200511102506.tar.gz",
           "LC082230672019091401T1-SC20200511102442.tar.gz",
           "LC082230672019093001T1-SC20200511102444.tar.gz",
           "LC082230672019120301T1-SC20200511102428.tar.gz",
           "LC082230682019062601T1-SC20200511105235.tar.gz"]
tile_path = 223
tile_row = 67

[datetime.datetime(2019, 8, 13, 0, 0), datetime.datetime(2019, 8, 29, 0, 0), datetime.datetime(2019, 9, 14, 0, 0), datetime.datetime(2019, 9, 30, 0, 0), datetime.datetime(2019, 12, 3, 0, 0), datetime.datetime(2019, 6, 26, 0, 0)]

In [46]:
archives_df = pd.DataFrame({"archives":archives,
                           "path":[int(archive[4:7]) for archive in archives],
                           "row":[int(archive[7:10]) for archive in archives],
                           "row":[int(archive[7:10]) for archive in archives],
                           "date":[datetime.datetime.strptime(archive[10:18], '%Y%m%d') for archive in archives]})
archives_df

,archives,path,row,date
0,LC082230672019081301T1-SC20200511102436.tar.gz,223,67,2019-08-13
1,LC082230672019082901T1-SC20200511102506.tar.gz,223,67,2019-08-29
2,LC082230672019091401T1-SC20200511102442.tar.gz,223,67,2019-09-14
3,LC082230672019093001T1-SC20200511102444.tar.gz,223,67,2019-09-30
4,LC082230672019120301T1-SC20200511102428.tar.gz,223,67,2019-12-03
5,LC082230682019062601T1-SC20200511105235.tar.gz,223,68,2019-06-26


In [13]:
ls_meta_df = pd.read_csv(meta_file)

In [14]:
ls_meta_df

,PANCHROMATIC_LINES,NADIR_OFFNADIR,sunAzimuth,REFLECTIVE_SAMPLES,upperLeftCornerLongitude,cloudCover,MAP_PROJECTION_L1,cartURL,sunElevation,path,...,UTM_ZONE,DATE_L1_GENERATED,GRID_CELL_SIZE_THERMAL,DATUM,COLLECTION_NUMBER,sceneID,RLUT_FILE_NAME,TIRS_SSM_MODEL,ROLL_ANGLE,receivingStation
0,15521,NADIR,-54.366833,7621,-33.75838,-1.00,UTM,https://earthexplorer.usgs.gov/download/extern...,-60.415141,91,...,25.0,2020/05/31,30,WGS84,1,LC80911802020152LGN00,LC08RLUT_20150303_20431231_01_12.h5,PRELIMINARY,0.000,LGN
1,18061,NADIR,-57.362968,9021,-71.42990,97.81,UTM,https://earthexplorer.usgs.gov/download/extern...,14.741300,91,...,18.0,2020/05/31,30,WGS84,1,LC80912382020152LGN00,LC08RLUT_20150303_20431231_01_12.h5,PRELIMINARY,-0.001,LGN
2,18281,NADIR,-61.178973,9131,-74.81578,80.96,UTM,https://earthexplorer.usgs.gov/download/extern...,16.085112,91,...,17.0,2020/05/31,30,WGS84,1,LC80912392020152LGN00,LC08RLUT_20150303_20431231_01_12.h5,PRELIMINARY,-0.001,LGN
3,18321,NADIR,-65.616001,9161,-78.76340,98.07,UTM,https://earthexplorer.usgs.gov/download/extern...,17.402916,91,...,16.0,2020/05/31,30,WGS84,1,LC80912402020152LGN00,LC08RLUT_20150303_20431231_01_12.h5,PRELIMINARY,-0.001,LGN
4,17241,NADIR,177.673927,8581,159.97525,39.30,UTM,https://earthexplorer.usgs.gov/download/extern...,41.024296,107,...,57.0,2020/05/31,30,WGS84,1,LC81070102020152LGN00,LC08RLUT_20150303_20431231_01_12.h5,PRELIMINARY,-0.001,LGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029995,15681,NADIR,-33.473545,7711,131.29138,-1.00,UTM,https://earthexplorer.usgs.gov/download/extern...,-29.167291,212,...,52.0,2017/03/24,30,WGS84,1,LC82122052016156LGN01,LC08RLUT_20150303_20431231_01_12.h5,FINAL,0.000,LGN
1029996,16321,NADIR,150.831794,8061,-42.21601,98.57,UTM,https://earthexplorer.usgs.gov/download/extern...,58.282362,228,...,24.0,2017/03/24,30,WGS84,1,LC82280242016156LGN01,LC08RLUT_20150303_20431231_01_12.h5,FINAL,-0.001,LGN
1029997,15661,NADIR,144.404472,7721,-43.92914,99.98,UTM,https://earthexplorer.usgs.gov/download/extern...,61.357687,228,...,23.0,2017/03/24,30,WGS84,1,LC82280272016156LGN01,LC08RLUT_20150303_20431231_01_12.h5,FINAL,-0.001,LGN
1029998,15741,NADIR,139.559682,7741,-44.94965,99.99,UTM,https://earthexplorer.usgs.gov/download/extern...,63.230484,228,...,23.0,2017/03/24,30,WGS84,1,LC82280292016156LGN01,LC08RLUT_20150303_20431231_01_12.h5,FINAL,-0.001,LGN


In [23]:
list(ls_meta_df.columns)

['PANCHROMATIC_LINES', 'NADIR_OFFNADIR', 'sunAzimuth', 'REFLECTIVE_SAMPLES', 'upperLeftCornerLongitude', 'cloudCover', 'MAP_PROJECTION_L1', 'cartURL', 'sunElevation', 'path', 'BPF_NAME_TIRS', 'THERMAL_LINES', 'GROUND_CONTROL_POINTS_MODEL', 'row', 'imageQuality1', 'REFLECTIVE_LINES', 'ELLIPSOID', 'GEOMETRIC_RMSE_MODEL', 'browseURL', 'browseAvailable', 'dayOrNight', 'CPF_NAME', 'DATA_TYPE_L1', 'THERMAL_SAMPLES', 'upperRightCornerLatitude', 'lowerLeftCornerLatitude', 'sceneStartTime', 'dateUpdated', 'sensor', 'PANCHROMATIC_SAMPLES', 'GROUND_CONTROL_POINTS_VERSION', 'LANDSAT_PRODUCT_ID', 'acquisitionDate', 'upperRightCornerLongitude', 'PROCESSING_SOFTWARE_VERSION', 'GRID_CELL_SIZE_REFLECTIVE', 'lowerRightCornerLongitude', 'lowerRightCornerLatitude', 'sceneCenterLongitude', 'COLLECTION_CATEGORY', 'GRID_CELL_SIZE_PANCHROMATIC', 'BPF_NAME_OLI', 'sceneCenterLatitude', 'CLOUD_COVER_LAND', 'lowerLeftCornerLongitude', 'GEOMETRIC_RMSE_MODEL_X', 'GEOMETRIC_RMSE_MODEL_Y', 'sceneStopTime', 'upperLeft

In [26]:
ls_meta_df[(ls_meta_df["path"]==tile_path) & (ls_meta_df["row"]==tile_row)][["acquisitionDate", "LANDSAT_PRODUCT_ID"]]

,acquisitionDate,LANDSAT_PRODUCT_ID
3237,2020-05-27,LC08_L1TP_223067_20200527_20200527_01_RT
15077,2020-05-11,LC08_L1TP_223067_20200511_20200526_01_T1
26740,2020-04-25,LC08_L1TP_223067_20200425_20200509_01_T1
38255,2020-04-09,LC08_L1TP_223067_20200409_20200422_01_T1
49829,2020-03-24,LC08_L1TP_223067_20200324_20200409_01_T1
...,...,...
976625,2016-08-20,LC08_L1TP_223067_20160820_20170322_01_T1
987608,2016-08-04,LC08_L1TP_223067_20160804_20180528_01_T1
998610,2016-07-19,LC08_L1TP_223067_20160719_20170323_01_T1
1009745,2016-07-03,LC08_L1TP_223067_20160703_20170323_01_T1


In [ ]:



raster_path = os.path.join(data_dir, 'ls', 'LC082230672019093001T1-SC20200511102444.tar.gz/LC08_L1TP_223067_20190930_20191017_01_T1_sr_band6.tif')

bands = ['sr_band2', 'sr_band3', 'sr_band4', 'sr_band5']
products = ['LC082230672019093001T1-SC20200511102444.tar.gz/LC08_L1TP_223067_20190930_20191017_01_T1',
            ]

raster_tifs=['LC08_L1TP_223067_20190930_20191017_01_T1_' + band + '.tif' for band in bands]

raster_tifs = [raster_tif for raster_tif in raster_tifs]

file_pattern = 'LC08223067.*tar.gz$'

In [ ]:

tiffs

In [ ]:
aoi_gdf = gpd.read_file(aoi)
for tiff in [tiffs[2]]:
    out_tiff = tiff.replace('.tif', "_aoi.tif")
    with rasterio.open(tiff) as raster:
        # show(raster)
        raster_profile = raster.profile
        mask = aoi_gdf.to_crs(crs=raster_profile['crs'].wkt).geometry
        raster_image, raster_transform = rasterio.mask.mask(raster, mask, crop=True, filled=False)
    
    raster_profile.update({"height":raster_image.shape[1],
                           "width":raster_image.shape[2],
                           "transform":raster_transform})
    
    with rasterio.open(out_tiff, "w", **raster_profile) as raster:
        raster.write(raster_image)


In [ ]:
pwd()

In [ ]:
raster_image

In [ ]:
raster_path

In [ ]:
with gzip.open(meta_file, mode="rt") as file:
    df = pd.read_csv(file) 
    

In [ ]:
df[(df['path'] == tile_path) & (df['row'] == tile_row)]

In [ ]:
df.columns

In [ ]:
archives = [os.path.join(data_dir, file) for file in os.listdir(data_dir) if re.match(file_pattern, file)]

In [ ]:
archives[0]

In [ ]:
tar:/home/takahisa/Documents/study/rsgis/data/ls/LC082230672019093001T1-SC20200511102444.tar.gz/LC08_L1TP_223067_20190930_20191017_01_T1_sr_band6.tif

In [ ]:
with rasterio.open(archives[0]) as 

In [ ]:
# class for rasterstack
class Rasterstack:
    def __init__(self):
        self.rasters = []
        self.aoi = None
    def clip(self):
        pass